# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/), [Valery Marchenkov](https://www.linkedin.com/in/vmarchenkoff/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages.

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for English and French languages.

In [1]:
import requests
import gzip
import shutil
import os
import numpy as np

In [2]:
def apply_mapping(X, W):
    return X.dot(W)

In [3]:
# !wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
# !gzip -d cc.en.300.vec.gz

# !wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
# !gzip -d cc.fr.300.vec.gz

In [4]:
def download_and_extract(url, output_path):
    gz_path = output_path + '.gz'
    
    if not os.path.exists(output_path):
        print(f"Downloading {url} ...")
        response = requests.get(url, stream=True)
        with open(gz_path, 'wb') as f:
            shutil.copyfileobj(response.raw, f)
        print("Download complete.")

        # Decompress file
        print("Extracting...")
        with gzip.open(gz_path, 'rb') as f_in:
            with open(output_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print("Extraction complete.")

        os.remove(gz_path)  # Optional: remove .gz file to save space
    else:
        print(f"{output_path} already exists.")

# en
download_and_extract(
    'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz',
    'cc.en.300.vec'
)

# fr
download_and_extract(
    'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz',
    'cc.fr.300.vec'
)

cc.en.300.vec already exists.
cc.fr.300.vec already exists.


After downloading and extracting the vectors, we should be able to load them using the [gensim](https://radimrehurek.com/gensim/) library:

In [5]:
from gensim.models import KeyedVectors
import numpy as np


en_emb = KeyedVectors.load_word2vec_format("cc.en.300.vec")
fr_emb = KeyedVectors.load_word2vec_format("cc.fr.300.vec")

Once you've loaded the vectors, you can use the `KeyedVectors` interface to get word embeddings and/or query most similar words by embedding:

In [6]:
august_embedding = en_emb["august"]
august_embedding.shape, august_embedding[:5]

((300,), array([-0.0522,  0.0364, -0.1252,  0.0053,  0.0382], dtype=float32))

In [7]:
en_emb.most_similar([august_embedding])

[('august', 1.0000001192092896),
 ('september', 0.825283944606781),
 ('october', 0.8111193180084229),
 ('june', 0.8050147891044617),
 ('july', 0.797055184841156),
 ('november', 0.7883636355400085),
 ('february', 0.783197283744812),
 ('december', 0.7824539542198181),
 ('january', 0.7743154764175415),
 ('april', 0.7621644735336304)]

The latter function also allows you to vary the amount of closest words via the `topn` argument:

In [8]:
en_emb.most_similar([august_embedding], topn=3)

[('august', 1.0000001192092896),
 ('september', 0.825283944606781),
 ('october', 0.8111193180084229)]

Another feature of `KeyedVectors` is that it allows to compute embeddings for multiple words simultaneously:

In [9]:
en_emb[["august", "september"]].shape

(2, 300)

Everything above is true for the embeddings for French language.

In [10]:
fr_emb.most_similar([fr_emb["aout"]])

[('aout', 1.0),
 ('Aout', 0.8249963521957397),
 ('juillet', 0.8109882473945618),
 ('fevrier', 0.8072442412376404),
 ('septembre', 0.7838520407676697),
 ('août', 0.7791767716407776),
 ('juin', 0.7692080736160278),
 ('octobre', 0.7597455382347107),
 ('decembre', 0.7595791220664978),
 ('avril', 0.7390779256820679)]

However, french and english embeddings were trained independently of each other. This means, that there is no obvious connection between values in embeddings for similar words in French and English:

In [11]:
fr_emb.most_similar([en_emb["august"]])

[('2003Pays', 0.23082852363586426),
 ('Montsoriu', 0.22505582869052887),
 ('2015Pays', 0.2221839874982834),
 ('2013Genre', 0.2095685601234436),
 ('AdiCloud', 0.201865091919899),
 ('Bagua', 0.20061466097831726),
 ('2003Paysans', 0.2001495510339737),
 ('ValenceLa', 0.2001475989818573),
 ('Luddites', 0.19998176395893097),
 ('Guadalquivir', 0.198755145072937)]

## Translation

We'll build a simple translator, which will try to predict the french embedding from the english one. For this we'll need a dataset of word pairs.

In [12]:
def load_word_pairs(filename):
    en_fr_pairs = []
    en_vectors = []
    fr_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            en, fr = line.rstrip().split(" ")
            if en not in en_emb or fr not in fr_emb:
                continue
            en_fr_pairs.append((en, fr))
            en_vectors.append(en_emb[en])
            fr_vectors.append(fr_emb[fr])
    return en_fr_pairs, np.array(en_vectors), np.array(fr_vectors)

We will train our model to predict embedding for the french word from embedding of its english counterpart. For this reason we split our train and test data into english and french words and compute corresponding embeddings to obtain `X` (english embeddings) and `y` (french embeddings).

In [13]:
#!wget -O en-fr.train.txt https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.train.txt
#!wget -O en-fr.test.txt https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.test.txt

In [14]:
def download_file(url, local_filename):
    print(f"Downloading {url} ...")
    response = requests.get(url)
    response.raise_for_status()

    with open(local_filename, 'wb') as f:
        f.write(response.content)
    print(f"Saved as {local_filename}")

#train 
download_file(
    'https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.train.txt',
    'en-fr.train.txt'
)

#test 
download_file(
    'https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.test.txt',
    'en-fr.test.txt'
)


Saved as en-fr.train.txt
Saved as en-fr.test.txt


In [15]:
en_fr_train, X_train, Y_train = load_word_pairs("en-fr.train.txt")
en_fr_test, X_test, Y_test = load_word_pairs("en-fr.test.txt")

In [16]:
en_fr_train[33:44]

[('which', 'laquelle'),
 ('which', 'lequel'),
 ('also', 'aussi'),
 ('also', 'egalement'),
 ('but', 'mais'),
 ('have', 'avoir'),
 ('have', 'ont'),
 ('one', 'un'),
 ('one', 'une'),
 ('one', 'one'),
 ('new', 'nouveau')]

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called [Procrustes problem](https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem):

$$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$$

or

$$W^*= \arg\min_W \|XW^T - Y\|_F$$

where $\|\cdot\|_F$ denotes Frobenius norm.

> **Note:** in second formula, $W$ and $x$ seem to have switched places. This happens because the $X$ matrix is composed of objects $x_i$ in *rows* not *columns*, i.e. it is kind of composed of $x_i^T$. This means that $X \in \mathbb{R}^{N \times D}$, where $N$ is the number of items and $D$ is the embedding dimensionality. The same is true for the $Y$.

$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$ looks like simple multiple linear regression without bias. The `sklearn` allows you to turn off the bias in `LinearRegression` via the `fit_intercept` argument (in fact they simply call bias the intercept). So let's code.

In [17]:
from sklearn.linear_model import LinearRegression


# YOUR CODE HERE
#

mapping_model = LinearRegression(fit_intercept=False)
mapping_model.fit(X_train, Y_train)

W = mapping_model.coef_
print(W.shape)

(300, 300)


Let's take a look at neigbours of the vector of word _"august"_ (_"aout"_ in French) after linear transform.

In [18]:
mapped_august = mapping_model.predict(en_emb["august"].reshape(1, -1))[0]
print("Ближайшее францусзкое 'august':")
for word, sim in fr_emb.most_similar([mapped_august], topn=5):
    print(f"  {word}: {sim:.4f}")


Ближайшее францусзкое 'august':
  aout: 0.7466
  juin: 0.7294
  juillet: 0.7228
  septembre: 0.7224
  mars: 0.7154


As quality measure we will use precision top-1, top-5 and top-10 (for each transformed english embedding we count how many right target pairs are found in top N nearest neighbours in french embedding space).

In [19]:
def precision(pairs, mapped_vectors, topn=1):
    correct = 0
    total = len(pairs)
    for i, (en_word, fr_word) in enumerate(pairs):
        vec = mapped_vectors[i]
        neighbors = fr_emb.most_similar([vec], topn=topn)
        neighbor_words = [word for word, sim in neighbors]
        if fr_word in neighbor_words:
            correct += 1
    return correct / total

In [20]:
assert precision([("august", "aout")], mapped_august.reshape(1, -1), topn=5) == 1.0
assert precision([("august", "aout")], mapped_august.reshape(1, -1), topn=9) == 1.0
assert precision([("august", "aout")], mapped_august.reshape(1, -1), topn=10) == 1.0

In [21]:
# assert precision([("august", "aout")], mapped_august, topn=5) == 1.0
# assert precision([("august", "aout")], mapped_august, topn=9) == 1.0
# assert precision([("august", "aout")], mapped_august, topn=10) == 1.0

mapped_test_vectors = mapping_model.predict(X_test)
precision_at1 = precision(en_fr_test[:100], mapped_test_vectors[:100], topn=1)
precision_at5 = precision(en_fr_test[:100], mapped_test_vectors[:100], topn=5)
precision_at10 = precision(en_fr_test[:100], mapped_test_vectors[:100], topn=10)
print(f"Precision фе 1: {precision_at1:.2f}")
print(f"Precision фе 5: {precision_at5:.2f}")
print(f"Precision фе 10: {precision_at10:.2f}")

Precision фе 1: 0.37
Precision фе 5: 0.61
Precision фе 10: 0.74


Note that our `precision` function accepts lists of pairs of words, whereas we have dataframes. However, it is not a problem: we can get a list (actually, numpy array) of pairs via the `values` property.

In [22]:
assert precision(en_fr_test[:100], X_test[:100]) == 0.0
assert precision(en_fr_test[:100], Y_test[:100]) == 1.0

In [23]:
U, s, Vt = np.linalg.svd(X_train.T.dot(Y_train))

W_orth = U.dot(Vt)

Let's see how well our model is doing.

In [24]:
precision_top1 = precision(en_fr_test[:100], apply_mapping(X_test[:100], W_orth), topn=1)
precision_top5 = precision(en_fr_test[:100], apply_mapping(X_test[:100], W_orth), topn=5)#

In [25]:
print(precision_top1)
print(precision_top5)

0.36
0.71


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$(W^T)^*= \arg\min_{W^T} \|XW^T - Y\|_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$(W^T)^*=UV^T$$

In [26]:
# YOUR CODE HERE


# U, s, Vt = np.linalg.svd(X_train.T.dot(Y_train))

# W_orth = U.dot(Vt)

print(W_orth.shape)

(300, 300)


Now our `mapping` is just a numpy array, meaning that it has no `predict` method. However, from the formulae above we know, that prediction is done using the matrix multiplication:

Now let's compute our precision values and see, whether our trick did improve the results.

In [27]:
fr_emb.most_similar([np.matmul(en_emb['august'], W_orth)])

[('aout', 0.6520753502845764),
 ('juin', 0.6361874938011169),
 ('juillet', 0.6292588114738464),
 ('septembre', 0.6282913684844971),
 ('octobre', 0.6213313341140747),
 ('mars', 0.6159636378288269),
 ('août', 0.6119704246520996),
 ('novembre', 0.6090866327285767),
 ('fevrier', 0.606033444404602),
 ('février', 0.6047893166542053)]

In [28]:
print(precision(en_fr_test[:100], np.matmul(X_test[:100], W_orth)))
print(precision(en_fr_test[:100], np.matmul(X_test[:100], W_orth), 5))

0.36
0.71


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Now let's translate these sentences word-by-word. Before that, however, don't forget to tokenize your sentences. For that you may (or may not) find the `nltk.tokenize.WordPunctTokenizer` to be very useful.

In [29]:
import nltk
nltk.download('punkt')
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\immor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
def translate(sentence):
    translated_tokens = []
    tokens = tokenizer.tokenize(sentence)
    for token in tokens:
        if not any(ch.isalpha() for ch in token):
            translated_tokens.append(token)
            continue
        if token in en_emb:
            vec = en_emb[token]
        elif token.lower() in en_emb:
            vec = en_emb[token.lower()]
        else:
            translated_tokens.append("<UNK>")
            continue
        mapped_vec = vec.dot(W_orth)
        fr_word = fr_emb.most_similar([mapped_vec], topn=1)[0][0]
        translated_tokens.append(fr_word)
    return " ".join(translated_tokens)


In [31]:
assert translate(".") == "."
assert translate("I walk around Paris") == "je marcher autour Paris"

In [32]:
print(translate("."))
print(translate("I walk around Paris"))

.
je marcher autour Paris


Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [33]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples
import re
import string

nltk.download('twitter_samples')
nltk.download('stopwords')

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        # if (word not in stopwords_english and  # remove stopwords
        #     word not in string.punctuation):  # remove punctuation
        if word not in string.punctuation:
            tweets_clean.append(word)
            # stem_word = stemmer.stem(word)  # stemming word
            # tweets_clean.append(stem_word)

    return " ".join(tweets_clean)

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\immor\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\immor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
twitter_samples.strings('positive_tweets.json')[10:15]

['#FollowFriday @wncer1 @Defense_gouv for being top influencers in my community this week :)',
 "Who Wouldn't Love These Big....Juicy....Selfies :) - http://t.co/QVzjgd1uFo http://t.co/oWBL11eQRY",
 '@Mish23615351  follow @jnlazts &amp; http://t.co/RCvcYYO0Iq follow u back :)',
 "@jjulieredburn Perfect, so you already know what's waiting for you :)",
 'Great new opportunity for junior triathletes aged 12 and 13 at the Gatorade series! Get your entries in :) http://t.co/of3DyOzML0']

In [35]:
for i in twitter_samples.strings('positive_tweets.json')[10:15]:
    print(i, process_tweet(i), sep='\n\n', end='\n-----------------\n')

#FollowFriday @wncer1 @Defense_gouv for being top influencers in my community this week :)

followfriday for being top influencers in my community this week :)
-----------------
Who Wouldn't Love These Big....Juicy....Selfies :) - http://t.co/QVzjgd1uFo http://t.co/oWBL11eQRY

who wouldn't love these big ... juicy ... selfies :)
-----------------
@Mish23615351  follow @jnlazts &amp; http://t.co/RCvcYYO0Iq follow u back :)

follow
-----------------
@jjulieredburn Perfect, so you already know what's waiting for you :)

perfect so you already know what's waiting for you :)
-----------------
Great new opportunity for junior triathletes aged 12 and 13 at the Gatorade series! Get your entries in :) http://t.co/of3DyOzML0

great new opportunity for junior triathletes aged 12 and 13 at the gatorade series get your entries in :)
-----------------


Your translation:

In [36]:
for i in twitter_samples.strings('positive_tweets.json')[:10]:
    print(i, process_tweet(i), translate(process_tweet(i)), sep='\n\n', end='\n-----------------\n')

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

followfriday for being top engaged members in my community this week :)

hashtag pour être top engagé membres dans mon communauté cette semaine :)
-----------------
@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!

hey james how odd :/ please call our contact centre on 02392441234 and we will be able to assist you :) many thanks

hey christopher comment bizarre :/ veuillez appeler notre contacter centre sur 02392441234 et nous sera être puisse amener aider vous :) nombreux merci
-----------------
@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!

we had a listen last night :) as you bleed is an amazing track when are you in scotland

nous avait un écouter dernier soir :) comme vous saigner est un incroyable track quand sont vous dans ecosse


Great! 